In [223]:
import pandas as pd
import numpy as np

In [224]:
df = pd.read_csv('dataset/train.csv')

In [225]:
df.shape

(36992, 25)

In [226]:
df.columns

Index(['customer_id', 'Name', 'age', 'gender', 'security_no',
       'region_category', 'membership_category', 'joining_date',
       'joined_through_referral', 'referral_id', 'preferred_offer_types',
       'medium_of_operation', 'internet_option', 'last_visit_time',
       'days_since_last_login', 'avg_time_spent', 'avg_transaction_value',
       'avg_frequency_login_days', 'points_in_wallet', 'used_special_discount',
       'offer_application_preference', 'past_complaint', 'complaint_status',
       'feedback', 'churn_risk_score'],
      dtype='object')

### Negative Churn Rate Fixing

In [227]:
# Cleaning the Null and Negative values
# df = df[df["churn_risk_score"]>=0]

# changinng negative values to zero
# df["churn_risk_score"] = np.where((df["churn_risk_score"] < 0),0,df["churn_risk_score"])
df[(df["churn_risk_score"]<0)]["churn_risk_score"] = int(0)
df[df["churn_risk_score"] > 5]["churn_risk_score"] = int(5)

<ipython-input-227-7a577168105a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[(df["churn_risk_score"]<0)]["churn_risk_score"] = int(0)


### joining_date_col convert to days to start

In [228]:
# Joining Date Columns Converted to Int
joining_date_col = pd.to_datetime(df["joining_date"])

In [229]:
df["joining_date"] = pd.to_datetime(df["joining_date"])  - joining_date_col.min()

In [230]:
# df["joining_date"].value_counts()
df["joining_date"] = df["joining_date"] / np.timedelta64(1, 'D')
df["joining_date"][0:10]

0    959.0
1    970.0
2    680.0
3    667.0
4    985.0
5    372.0
6     77.0
7    558.0
8    713.0
9    699.0
Name: joining_date, dtype: float64

In [231]:
# last_visit_time col here
pd.to_datetime(df["last_visit_time"],format="%H:%M:%S")[0:20]

0    1900-01-01 16:08:02
1    1900-01-01 12:38:13
2    1900-01-01 22:53:21
3    1900-01-01 15:57:50
4    1900-01-01 15:46:44
5    1900-01-01 06:46:07
6    1900-01-01 11:40:04
7    1900-01-01 07:52:43
8    1900-01-01 06:50:10
9    1900-01-01 19:10:16
10   1900-01-01 05:57:20
11   1900-01-01 11:56:11
12   1900-01-01 04:42:50
13   1900-01-01 08:35:05
14   1900-01-01 21:01:43
15   1900-01-01 13:48:22
16   1900-01-01 20:58:28
17   1900-01-01 03:42:26
18   1900-01-01 15:39:05
19   1900-01-01 11:46:54
Name: last_visit_time, dtype: datetime64[ns]

### last_visit_ns -> Last Visit Column to Float

In [232]:
# pd.TimeStamp(pd.to_datetime(df["last_visit_time"],format="%H:%M:%S"))
last_visit_ns = pd.to_datetime(df["last_visit_time"],format="%H:%M:%S")

In [233]:
ts = (last_visit_ns - np.datetime64('1900-01-01T00:00:00Z')) / np.timedelta64(1, 's')

<ipython-input-233-3235a64cf489>:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  ts = (last_visit_ns - np.datetime64('1900-01-01T00:00:00Z')) / np.timedelta64(1, 's')


In [234]:
ts

0        58082.0
1        45493.0
2        82401.0
3        57470.0
4        56804.0
          ...   
36987    15245.0
36988    83911.0
36989    13825.0
36990    35403.0
36991     5992.0
Name: last_visit_time, Length: 36992, dtype: float64

In [235]:
df["last_visit_time"] = ts

In [236]:
df = df.dropna()

In [237]:
df.shape

(28373, 25)

In [238]:
df.isna().count()

customer_id                     28373
Name                            28373
age                             28373
gender                          28373
security_no                     28373
region_category                 28373
membership_category             28373
joining_date                    28373
joined_through_referral         28373
referral_id                     28373
preferred_offer_types           28373
medium_of_operation             28373
internet_option                 28373
last_visit_time                 28373
days_since_last_login           28373
avg_time_spent                  28373
avg_transaction_value           28373
avg_frequency_login_days        28373
points_in_wallet                28373
used_special_discount           28373
offer_application_preference    28373
past_complaint                  28373
complaint_status                28373
feedback                        28373
churn_risk_score                28373
dtype: int64

In [239]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [240]:
df.dtypes

customer_id                      object
Name                             object
age                               int64
gender                           object
security_no                      object
region_category                  object
membership_category              object
joining_date                    float64
joined_through_referral          object
referral_id                      object
preferred_offer_types            object
medium_of_operation              object
internet_option                  object
last_visit_time                 float64
days_since_last_login             int64
avg_time_spent                  float64
avg_transaction_value           float64
avg_frequency_login_days         object
points_in_wallet                float64
used_special_discount            object
offer_application_preference     object
past_complaint                   object
complaint_status                 object
feedback                         object
churn_risk_score                  int64


In [241]:
df2 = df.select_dtypes(include="object").apply(label_encoder.fit_transform)

In [242]:
df2.head()

customer_id   Name  gender  security_no  region_category  \
0        24720  22051       0        26668                2   
2         4529  19480       0         1144                1   
3        24843   8084       1        24890                0   
4         4653  19011       0        22828                0   
5        26213  15752       1        20385                0   

   membership_category  joined_through_referral  referral_id  \
0                    3                        1         9603   
2                    2                        2          405   
3                    2                        2         4654   
4                    2                        1         9603   
5                    1                        1         9603   

   preferred_offer_types  medium_of_operation  internet_option  \
0                      1                    0                2   
2                      1                    2                2   
3                      1                    2                1   
4                      0                    3                1   
5                      1                    0                2   

   avg_frequency_login_days  used_special_discount  \
0                       570                      1   
2                       598                      0   
3                      1251                      0   
4                       567                      0   
5                       601                      1   

   offer_application_preference  past_complaint  complaint_status  feedback  
0                             1               0                 1         4  
2                             1               1                 3         3  
3                             1               1                 4         3  
4                             1               1                 2         3  
5                             0               1                 4         0

In [243]:
from sklearn.model_selection import train_test_split
X = df2[['gender',
       'region_category', 'membership_category', 
       'joined_through_referral', 'preferred_offer_types',
       'medium_of_operation', 'internet_option',
       'avg_frequency_login_days', 'used_special_discount',
       'offer_application_preference', 'past_complaint', 'complaint_status',
       'feedback']]
X[['points_in_wallet', 'age', 'avg_time_spent', 'last_visit_time', 'joining_date', 'days_since_last_login', 'avg_transaction_value']] = df[['points_in_wallet', 'age', 'avg_time_spent', 'last_visit_time', 'joining_date', 'days_since_last_login', 'avg_transaction_value']]
y = df["churn_risk_score"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# print(X.shape,y.shape)

C:\Users\MAYANK\anaconda3\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [244]:
X.head()

gender  region_category  membership_category  joined_through_referral  \
0       0                2                    3                        1   
2       0                1                    2                        2   
3       1                0                    2                        2   
4       0                0                    2                        1   
5       1                0                    1                        1   

   preferred_offer_types  medium_of_operation  internet_option  \
0                      1                    0                2   
2                      1                    2                2   
3                      1                    2                1   
4                      0                    3                1   
5                      1                    0                2   

   avg_frequency_login_days  used_special_discount  \
0                       570                      1   
2                       598                      0   
3                      1251                      0   
4                       567                      0   
5                       601                      1   

   offer_application_preference  past_complaint  complaint_status  feedback  \
0                             1               0                 1         4   
2                             1               1                 3         3   
3                             1               1                 4         3   
4                             1               1                 2         3   
5                             0               1                 4         0   

   points_in_wallet  age  avg_time_spent  last_visit_time  joining_date  \
0            781.75   18          300.63          58082.0         959.0   
2            500.69   44          516.16          82401.0         680.0   
3            567.66   37           53.27          57470.0         667.0   
4            663.06   31          113.13          56804.0         985.0   
5            722.27   13          433.62          24367.0         372.0   

   days_since_last_login  avg_transaction_value  
0                     17               53005.25  
2                     14               21027.00  
3                     11               25239.56  
4                     20               24483.66  
5                     23               13884.77

In [245]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [246]:
y_train.isna().sum()

0

In [247]:
y_train[y_train > 5]

Series([], Name: churn_risk_score, dtype: int64)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score,mean_squared_error
score = f1_score(y_test, y_pred,average="macro")
score

In [ ]:
result_df = pd.DataFrame({
    "churn_risk_score":y_pred.astype(int)
})

In [ ]:
result_df

In [ ]:
real_test_data = pd.read_csv("dataset/test.csv")
real_test_data.head()

In [ ]:
real_test_data2 = real_test_data[['age', 'gender',
       'region_category', 'membership_category', 'joining_date',
       'joined_through_referral', 'preferred_offer_types',
       'medium_of_operation', 'internet_option', 'last_visit_time',
       'days_since_last_login', 'avg_time_spent', 'avg_transaction_value',
       'avg_frequency_login_days', 'points_in_wallet', 'used_special_discount',
       'offer_application_preference', 'past_complaint', 'complaint_status',
       'feedback']]


In [ ]:
real_test_data2 = real_test_data2.fillna(real_test_data2.mean())

In [ ]:
# joining_date column conversions

In [ ]:
# Joining Date Columns Converted to Int
joining_date_col = pd.to_datetime(real_test_data2["joining_date"])

In [ ]:
real_test_data2["joining_date"] = pd.to_datetime(real_test_data2["joining_date"])  - joining_date_col.min()

In [ ]:
# df["joining_date"].value_counts()
real_test_data2["joining_date"] = real_test_data2["joining_date"] / np.timedelta64(1, 'D')
real_test_data2["joining_date"][0:10]

In [ ]:
real_test_data2.dtypes

### Last Visit Time Fix for Test

In [ ]:
# pd.TimeStamp(pd.to_datetime(df["last_visit_time"],format="%H:%M:%S"))
last_visit_ns = pd.to_datetime(real_test_data2["last_visit_time"],format="%H:%M:%S")

In [ ]:
ts = (last_visit_ns - np.datetime64('1900-01-01T00:00:00Z')) / np.timedelta64(1, 's')

In [ ]:
ts

In [ ]:
real_test_data2["last_visit_time"] = ts

In [ ]:
real_test_data2.dtypes

In [ ]:
encoded_str = real_test_data2.select_dtypes(include="object")#.apply(label_encoder.fit_transform)

In [ ]:
model_ip = encoded_str.astype(str).apply(label_encoder.fit_transform)

In [ ]:
model_ip[['points_in_wallet', 'age', 'avg_time_spent', 'last_visit_time', 'joining_date', 'days_since_last_login', 'avg_transaction_value']] = real_test_data2[['points_in_wallet', 'age', 'avg_time_spent', 'last_visit_time', 'joining_date', 'days_since_last_login', 'avg_transaction_value']].astype(str)
model_ip[['last_visit_time', 'joining_date']] = real_test_data2[['last_visit_time', 'joining_date']]
model_ip.head()

In [ ]:
model_ip[['last_visit_time', 'joining_date']].isna().sum()

In [ ]:
model_ip.dtypes

In [ ]:
# str_needed = real_test_data.select_dtypes(exclude=["float64","int64"])
real_test_pred = model.predict(model_ip)

In [ ]:
final_result_df = pd.DataFrame({
    "customer_id":real_test_data["customer_id"],
    "churn_risk_score":real_test_pred.astype(int)
})

In [ ]:
final_result_df.head()

In [ ]:
final_result_df.to_csv(f"FINAL_MODEL-score-{ score }.csv")

In [ ]:
print(score)